# Module 8: Vision-Based Car Following Cruise Control

Here we will combine multiple aspects of what we've learned to have our Jetbot follow another Jetbot. The goal is to have our robot detect an AprilTag attached to the back of another vehicle, then use a PD Controller to follow the position of the robot.

This module should follow Module 7: PID Control for Line Tracking

Below is our normal starter code for initializing all variables and packages. An image should appear to ensure it is working properly. Make sure to have the various values for your camera coefficient matrix, the side length of the AprilTags, and determine the distance behind the Jetbot you want to stay behind the other vehicle (in meters).

In [ ]:
import threading
import time
import cv2
import numpy as np
from jetbot import Robot, Camera, bgr8_to_jpeg
from IPython.display import display
import ipywidgets as widgets
import dt_apriltags

# Initialize JetBot
robot = Robot()
camera = Camera.instance()
detector = dt_apriltags.Detector()

From Module 3, you should have your intrinsic parameters for your camera and the length of the AprilTags. Make sure to add those values here.

Additionally, we also want to set a desired distance at which the <b><span style="color:#154734">JetBot</span></b> will follow behind the other vehicle. Set a distance you think would be appropriate in meters.

In [ ]:
# TODO: Change/Insert values here
cam_mtx = [94.861, 119.511, 119.506, 114.969]  # [fx, fy, cx, cy]
AT_size = 0.065  # meters
desired_distance = 0.1  # meters to maintain from tag

### Pipeline

Similar to our line tracking, we are creating a pipeline from when our <b><span style="color:#154734">JetBot</span></b> takes the image to the commands sent to the motors.

1. AprilTag Detection and Pose Estimation

    After we collect our camera frame, we want to check for any AprilTags in the image. If one is found, it is returned along with the translation matrix. There are two pieces of information we want to take into account from there, the x and z values. As we've discussed, the x represents how far to the left and right the AprilTag is with respect to the camera, while z represents the distance away.

2. Error Calculation

    To have this function, we want the AprilTag to be in the center and the desired distance away that we set above. But for PID Controllers, we need to input the error between the desired and actual. So we must calculate what the error would be for those values. Below are these equations:

    $$ x_{error}=-x_{offset}
    \\[2ex]
    z_{error}=z_{distance}-desired\_distance
    $$

3. PD Controller

    Since we have two variables and actions we are trying to account for (forward speed and turning), we want to create two separate PD Controllers for each. Each one will have their own separate P and D tuning variables.

4. Motor Mapping

    Finally, after we receive the controller output for speed and turn, we want to map those values to the motors so that the <b><span style="color:#154734">JetBot</span></b> is able to follow the vehicle with the AprilTag.

In [ ]:
# PD Controller Class
class PIDController:
    def __init__(self, Kp=0.4, Kd=0.2):
        self.Kp = Kp
        self.Kd = Kd
        self.last_error = 0.0
        self.last_time = time.time()

    def compute(self, error):
        current_time = time.time()
        dt = current_time - self.last_time
        de = (error - self.last_error) / dt if dt > 0 else 0.0
        output = self.Kp * error + self.Kd * de
        self.last_error = error
        self.last_time = current_time
        return output

# Widgets (live feed, distance, control output)
image_widget = widgets.Image(format='jpeg', width=300, height=300)
dist_label = widgets.Label(value="Translation Z: ---")
control_label = widgets.Label(value="Control Output: ---")

# Distance PID Widgets
kp_dist = widgets.FloatText(value=0.4, description='Kp Dist')
kd_dist = widgets.FloatText(value=0.2, description='Kd Dist')

# Angle PID Widgets
kp_angle = widgets.FloatText(value=0.6, description='Kp Angle')
kd_angle = widgets.FloatText(value=0.2, description='Kd Angle')

# Control buttons
go_button = widgets.Button(description="Go", button_style='success')
stop_button = widgets.Button(description="Stop", button_style='danger')
robot_enabled = False

def on_go_clicked(b):
    global robot_enabled
    robot_enabled = True

def on_stop_clicked(b):
    global robot_enabled
    robot_enabled = False
    robot.stop()

go_button.on_click(on_go_clicked)
stop_button.on_click(on_stop_clicked)

# Display widgets
display(widgets.VBox([
    image_widget,
    widgets.HBox([go_button, stop_button]),
    dist_label,
    control_label,
    widgets.Label(value='Distance PID:'),
    kp_dist, kd_dist,
    widgets.Label(value='Angle PID:'),
    kp_angle, kd_angle
]))

# PD Controllers
distance_pid = PIDController(Kp=kp_dist.value, Kd=kd_dist.value)
angle_pid = PIDController(Kp=kp_angle.value, Kd=kd_angle.value)

stop_event = threading.Event()

# AprilTag Tracking Loop
def follow_apriltag():
    global robot_enabled
    while not stop_event.is_set():
        try:
            frame = np.array(camera.value)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            detections = detector.detect(gray, estimate_tag_pose=True, 
                                         camera_params=cam_mtx, tag_size=AT_size)

            if detections:
                t_last_detection = time.time()
                detection = detections[0]
                pose_t = detection.pose_t

                z_distance = pose_t[2][0]
                x_offset = pose_t[0][0]  # left-right position
                dist_label.value = f"Translation Z: {z_distance:.3f} m"

                # Error Calculation
                # TODO: Add the equations from above to calculate the error.
                z_error = z_distance - desired_distance
                x_error = -x_offset

                # Update PID gains from sliders
                distance_pid.Kp = kp_dist.value
                distance_pid.Kd = kd_dist.value
                angle_pid.Kp = kp_angle.value
                angle_pid.Kd = kd_angle.value

                # Compute control outputs
                speed = distance_pid.compute(z_error)
                turn = angle_pid.compute(x_error)

                # Clamp speed to [0, 0.3] to prevent backing up
                speed = max(min(speed, 0.3), 0.0) # Remove this line to keep reversing
                turn = max(min(turn, 0.1), -0.1)

                control_label.value = f"Speed: {speed:.3f}, Turn: {turn:.3f}"

                if robot_enabled and z_distance > desired_distance:
                    left = max(min(speed - turn, 0.3), -0.3)
                    right = max(min(speed + turn, 0.3), -0.3)
                    robot.set_motors(left, right)
                else:
                    robot.stop()

                # Draw tag visual
                corners = detection.corners.astype(int)
                cv2.polylines(frame, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)
                cv2.putText(frame, f"ID: {detection.tag_id}", (corners[0][0], corners[0][1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            else:
                if time.time() - t_last_detection > 0.3:
                    dist_label.value = "Translation Z: ---"
                    control_label.value = "Control Output: ---"
                    robot.stop()

            image_widget.value = bgr8_to_jpeg(frame)

        except Exception as e:
            dist_label.value = f"Error: {str(e)}"
            control_label.value = "Control Output: ---"
            robot.stop()

        time.sleep(0.05)

# Starts the Feed
camera.start()
thread = threading.Thread(target=follow_apriltag, daemon=True)
thread.start()


In [ ]:
stop_event.set()
thread.join()
stop_event.clear()
camera.stop()